<div class="alert alert-block alert-info">
<b>Sentence BERT를 이용한 내용 기반 국문 저널 추천 시스템</b>
</div>

# III. Gompertz Function 적용 모델

---
# Setting

In [278]:
# setting path
import sys
sys.path.append('..') # parent directory 경로 추가

from common import *
from my import *

%matplotlib inline
%config InlineBackend.figure_format='retina'

# random seed 고정 
import os, random
def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
#     tf.random.set_seed(seed) # Tensorflow 사용시 
SEED = 777
set_seeds(SEED)

In [279]:
import os
import pickle
from tqdm import tqdm

import collections
from heapq import nlargest
from operator import itemgetter
from joblib import Parallel, delayed, parallel_backend

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt, Mecab

import math
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

---
# Train 데이터 관련 준비

In [260]:
# 전처리한 train 데이터(dataframe) 다시 읽어오기
train = pd.read_csv('./data/train.csv', index_col=0)
print(train.shape)

# Train y 
train_y = train['journal'] # Train 데이터 document id - journal
print(train_y.shape)

# Train 데이터셋 기준 각 document id - index 매핑 (데이터셋에서의 위치)
train_id_index = pd.Series(range(len(train.index)), index=train.index)
print(len(train_id_index))

# Train SBERT embedding npy 파일 다시 읽기
train_embed = np.load('./data/train_embed.npy')
print(train_embed.shape)

(92760, 7)
(92760,)
92760
(92760, 768)


In [92]:
# 저널명 - Index 매핑  (저널 고정 순서)
journ_order = train.groupby('journal').groups.keys()
journ_index = pd.Series(range(len(journ_order)), index=journ_order)
print(journ_index[:5])
print(journ_index[-5:])

journal
CRM연구                                                      0
Child Health Nursing Research                              1
Clinical and Experimental Reproductive Medicine            2
Clinics in Shoulder and Elbow                              3
Communications for Statistical Applications and Methods    4
dtype: int64
journal
해양환경안전학회지    263
혜화의학회지       264
화약ㆍ발파        265
환경영향평가       266
환경정책연구       267
dtype: int64


In [261]:
from sklearn.feature_extraction.text import CountVectorizer

# Journal-Title Matrix (JTM)
# journal별 title CountVectorize
train_journ_tt = train.groupby('journal')['title_nn'].apply(' '.join) # journal별로 모든 논문들 title text 합치기
count_vect_tt = CountVectorizer(min_df=1, ngram_range=(1,1)) # unigram
jtm = count_vect_tt.fit_transform(train_journ_tt) # sparse matrix
print(jtm.shape)

# csr matrix -> dataframe
# jtm_df = pd.DataFrame(jtm.toarray(), index=train_journ_tt.index, columns=count_vect_tt.get_feature_names_out())

# ================================================= #
# Journal-Keyword Matrix (JKM)
# journal별 keyword CountVectorize
train_journ_kw = train.groupby('journal')['keyword'].apply(' '.join) # journal별로 모든 논문들 keyword text 합치기
count_vect_kw = CountVectorizer(min_df=1, ngram_range=(1,1)) # unigram
jkm = count_vect_kw.fit_transform(train_journ_kw) # sparse matrix
print(jkm.shape)

# csr matrix -> dataframe
# jkm_df = pd.DataFrame(jkm.toarray(), index=train_journ_kw.index, columns=count_vect_kw.get_feature_names_out())

(268, 43041)
(268, 135402)


---
# Test 데이터 관련 준비

In [264]:
# 전처리한 test 데이터(dataframe) 다시 읽어오기
test = pd.read_csv('./data/test.csv', index_col=0)
print(test.shape)

# Test y
test_y = test['journal'] # Test 데이터 document id - journal
print(test_y.shape)

# Test 데이터셋 기준 document id - index 매핑 (데이터셋에서의 위치)
test_id_index = pd.Series(range(len(test.index)), index=test.index)
print(len(test_id_index))

# Test SBERT embedding npy 파일 다시 읽기
test_embed = np.load('./data/test_embed.npy')
print(test_embed.shape)

(10307, 7)
(10307,)
10307
(10307, 768)


## 문서 vs. 문서 유사도 (S1)
- Test 데이터 사용
- Test 데이터의 SBERT 임베딩된 Abstract vs. Train 데이터의 SBERT 임베딩된 Abstract -> 코사인 유사도 계산

In [265]:
# test document의 embedding vectors & train document의 embedding vectors 코사인 유사도 분석
from sklearn.metrics.pairwise import cosine_similarity

s1 = cosine_similarity(test_embed, train_embed) # test embed vs. train embed
print(s1.shape)
s1[:5]

(10307, 92760)


array([[0.5445481 , 0.33625972, 0.1614262 , ..., 0.3330929 , 0.557525  ,
        0.31247985],
       [0.6503759 , 0.45653108, 0.29908445, ..., 0.50919765, 0.6316433 ,
        0.48862016],
       [0.31182605, 0.34367323, 0.24949932, ..., 0.44575486, 0.24206194,
        0.45320866],
       [0.42790985, 0.48927057, 0.2149891 , ..., 0.47800317, 0.53168607,
        0.46760318],
       [0.46252072, 0.38382778, 0.40913084, ..., 0.26628563, 0.23586869,
        0.44414553]], dtype=float32)

In [8]:
# numpy 배열 파일로 저장
# np.save('s1', s1)

# 저장한 npy 파일 다시 읽기
# s1 = np.load('s1.npy')

## Customized Function | List of Journals (R)

- 곰페르츠 함수(Gompertz function) 적용
    - S-curved 형태가 됨
- 저널별 최종 점수 계산 방법:
    - 저널별로 위 유사도 점수를 평균(mean)

In [39]:
import math
import collections
from heapq import nlargest
from operator import itemgetter
from joblib import Parallel, delayed, parallel_backend

def get_all_sim_journ(doc_sims, threshold=0.7):
    '''
    doc_sims : numpy array; 각 문서vs.문서 유사도 점수
    threshold : "높은 유사도"의 기준
    '''
    # Gompertz function을 사용하여 similarity score 계산
    a = 0
    b = -0.1
    c = 100
    d = 60
    sim_weighted = a + (c-a) * (1-math.exp(1)**(-math.exp(1)**(-b*(doc_sims*100-d)))) # 문서별 문서vs.문서 유사도
    
    # 각 점수마다 해당 document id 붙이기 ([document id, weighted 유사도 점수] 형태로)
    sim_weighted = list(zip(train_id_index.index, sim_weighted)) # e.g. ('JAKO201610364779000', 0.5445481)
    
    # weighted 유사도 >= threshold
    #sim_weighted = list(filter(lambda x: x[1] >= threshold, sim_weighted))
    
    # [document id, 유사도 점수] -> 저널별 [journal 이름, 유사도 총 점수]
    sum_dict = collections.defaultdict(list)
    for doc_id, sim_score in sim_weighted:
        journ_name = train_y[doc_id] # 저널명 가져오기
        sum_dict[journ_name].append(sim_score) # 저널별 유사도 점수 합치기
    
    mean_dict = {key: np.mean(values) for key, values in sum_dict.items()}
    
    # 총 유사도 점수가 높은 순서대로 저널 이름 정렬 (S-Sort)
    journ_sorted = list(dict(sorted(mean_dict.items(), key=itemgetter(1), reverse=True)).keys())
    
    return journ_sorted

# 추천 후보 저널 - list of journals (R)
workers = os.cpu_count() * 2
with parallel_backend(backend='loky', n_jobs=workers):
    r = list(Parallel()(delayed(get_all_sim_journ)(doc_sims) for doc_sims in tqdm(s1, position=0, leave=True)))

print(len(r))
print(r[0][:10])

100%|█████████████████████████████████████| 10307/10307 [07:31<00:00, 22.81it/s]


10307
['한국가금학회지', '대한한방부인과학회지', 'Journal of Animal Science and Technology', '원예과학기술지', '농약과학회지', 'Weed & Turfgrass Science', '농업과학연구', '시설원예ㆍ식물공장', 'Clinical and Experimental Reproductive Medicine', 'Radiation Oncology Journal']


In [14]:
import pickle

# 유사도 높은 journal 리스트 pickle로 저장
# with open('./data/r_gompertz.pkl', 'wb') as f:
#     pickle.dump(r, f)

# 저장한 pickle 다시 읽기
# with open('./data/r_gompertz.pkl', 'rb') as f:
#     r = pickle.load(f)

----
# # Phase 2: 후보 저널 재정렬
- 후보 저널 재정렬
    - 입력 문서 vs. 저널의 Title, Keyword 유사도 기반 (S2+S3)
    - 초록 외 키워느나 제목이 입력된 경우, 저널에 출판된 문서들(train)의 키워드와 일치도를 계산하여 추천 결과를 개선하기 위한 과정
    - 후보 저널 리스트업 된 것에 순위 sort하기 위한 과정

## 문서 vs. 저널 유사도 (S2+S3)
- Keyword 유사도 (S2) + Title 유사도 (S3)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# ================================================= #
# test document vs. train journal의 keyword 유사도 구하기 (S2)

# 각 Test 데이터의 문서별 Keyword Matrix 생성 (Test JKM)
test_doc_kw = test['keyword']
print(test_doc_kw.shape)

# document-keyword matrix 생성 (JKM 생성때 fit된 count vectorizer 사용)
test_doc_kw_mat = count_vect_kw.transform(test_doc_kw)
print(test_doc_kw_mat.shape)

# dataframe 형태로 확인
# d_kw_df = pd.DataFrame(d_kw_mat.toarray(), index=d_kw.index, columns=count_vect_kw.get_feature_names_out())

# keyword 사이의 document vs. journal 유사도 계산
s2 = cosine_similarity(test_doc_kw_mat, jkm)
print(s2.shape)

# ================================================= #
# test document vs. train journal의 title 유사도 구하기 (S3)

# 각 Test 데이터의 문서별 Title Matrix 생성 (Test JTM)
test_doc_tt = test['title_nn']
print(test_doc_tt.shape)

# document-title matrix 생성 (JTM 생성때 fit된 count vectorizer 사용)
test_doc_tt_mat = count_vect_tt.transform(test_doc_tt)
print(test_doc_tt_mat.shape)

# dataframe 형태로 확인
# d_tt_df = pd.DataFrame(d_tt_mat.toarray(), index=d_tt.index, columns=count_vect_tt.get_feature_names_out())

# title 사이의 document vs. journal 유사도 계산
s3 = cosine_similarity(test_doc_tt_mat, jtm)
print(s3.shape)

# ================================================= #
# Title 코사인 유사도 + Keyword 코사인 유사도 더하기 (S2+S3)
s23 = s2 + s3
print(s23.shape)
s23[:5]

(10307,)
(10307, 135402)
(10307, 268)
(10307,)
(10307, 43041)
(10307, 268)
(10307, 268)


array([[0.16510017, 0.09318388, 0.07643517, ..., 0.03647989, 0.12525107,
        0.07358115],
       [0.        , 0.        , 0.        , ..., 0.0032244 , 0.00759136,
        0.00520297],
       [0.1249516 , 0.06112054, 0.05013484, ..., 0.19781094, 0.13443356,
        0.11188206],
       [0.        , 0.        , 0.        , ..., 0.03224398, 0.01180878,
        0.00346865],
       [0.11973687, 0.12910096, 0.07206377, ..., 0.17196787, 0.17375783,
        0.1314451 ]])

## 추천 저널 리스트 재배치

**Note: 코드가 실행되는 곳은 아래 "Phase"의 모델링 부분에서 실행 됨**

In [ ]:
# 문서 vs. 저널 유사도(S2+S3)에 따른 후보 저널 재배치

from operator import itemgetter

# def resort_r(tt_kw_s, i):
#     # doc-journal별 유사도 점수에 journal 이름 붙이기 
#     journ_scores = list(zip(journ_index.index, tt_kw_s[i])) # journal 이름 붙이기
#     # 높은 유사도순으로 journal 정렬
#     journ_scores_sorted = sorted(journ_scores, key=lambda x: x[1], reverse=True) 
#     # journal이 기존 list of journals에 있는 journal이라면 포함 (정렬된 순서에 맞춰 새로운 R 생성)
#     journ_filtered = [journ for journ, sim in journ_scores_sorted if journ in top_k_journs[i]]
#     return journ_filtered

def resort_r(tt_kw_s, top_k_journs, i):
    # 각 doc의 journal별 title+keyword 유사도 점수에 journal 이름 붙이기 
    journ_scores = dict(zip(journ_index.index, tt_kw_s[i])) # journal 이름 붙이기
    # 높은 유사도순으로 journal 재배치 (기존 list of journals에 있는 저널만 포함) -> 새로운 R 생성
    journ_sorted = list(dict(sorted(filter(lambda x: x[0] in top_k_journs[i], journ_scores.items()), key=itemgetter(1), reverse=True)).keys())
    return journ_sorted

---
# # Phase 3: 후보 저널 추가
- Test 데이터셋 사용
- 후보 저널 추가
    - 저널 vs. 저널 유사도 (S4) 기반
    - Title, Keyword 유사도
    - 새로 추가할 수 있는, 상위 후보 저널과 유사한 후보 저널을 탐색

## 저널 vs. 저널 유사도 (S4) | 후보 저널 추가

In [343]:
import collections
from heapq import nlargest
from operator import itemgetter
from sklearn.metrics.pairwise import cosine_similarity
from joblib import Parallel, delayed, parallel_backend

# 저널 vs. 저널 유사도 (S4) 계산 및 후보 저널 추가
def find_other_sim_journ(cur_top_journs):
    '''
    # Description :
    # 추천 저널 후보 리스트에서, 맨 앞의 최상위 저널(R1) 1개에 대해 저널들과의 유사도를 분석 (저널 vs. 저널 유사도 분석)
    # 이후, 유사도 높은 저널부터 순차적으로 추천 저널 후보 리스트에 없는 저널을 찾기
    # 만약 찾았지만, 발견한 저널의 유사도 점수가 0보다 크지 않다면 (그 뒤 후보들의 유사도는 볼 필요 없이)
    # R1은 pass하고 다음 상위 저널(R2, R3, ... 순차적으로)로 같은 방법으로 탐색
    '''
    def if_not_in_r(x):
        # list of journals에 이미 있는지/없는지 확인
        if x[0] in cur_top_journs:
            return False
        else:
            return True
    
    # list of journals이 애초에 비어있는 경우
    if not len(cur_top_journs):
        # 빈 list of journals 그대로 반환
        return cur_top_journs
    
    # 추가 후보 저널 탐색
    add_journ_found = False
    for i in range(len(cur_top_journs)):
        # 현재 선택된 상위 저널
        best_journ = cur_top_journs[i] # 맨 앞의(최상위) 후보 저널부터 하나씩

        # # 해당 상위 저널과 다른 저널들의 title 유사도 구하기
        # best_j_tt = train[train['journal'] == best_journ].groupby('journal')['title_nn'].apply(' '.join) # 해당 상위 저널의 (모든 doc의) 전처리된 title text 모으기
        # best_j_tt_mat = count_vect_tt.transform(best_j_tt) # 상위 저널의 journal-title matrix 생성 (train JTM 생성시 fit한 count vectorizer로 transform)
        best_j_tt_mat = jtm[journ_index[best_journ]]
        best_tt_sim = cosine_similarity(best_j_tt_mat, jtm) # 상위 저널 JTM - train JTM 과의 title 코사인 유사도 (1, 268)

        # 해당 상위 저널과 다른 후보 저널들의 keyword 유사도 구하기
        # best_j_kw = train[train['journal'] == best_journ].groupby('journal')['keyword'].apply(' '.join) # 해당 상위 저널의 (모든 doc의) 전처리된 keyword text 모으기
        # best_j_kw_mat = count_vect_kw.transform(best_j_kw) # 상위 저널의 journal-keyword matrix 생성 (train JKM 생성시 fit한 count vectorizer로 transform)
        best_j_kw_mat = jkm[journ_index[best_journ]]
        best_kw_sim = cosine_similarity(best_j_kw_mat, jkm) # 상위 저널 JKM - train JKM 과의 keyword 코사인 유사도

        # 해당 후보 저널의 다른 저널들과의 최종 유사도 구하기
        s4 = best_tt_sim + best_kw_sim # Title 코사인 유사도 + Keyword 코사인 유사도 더하기
        s4 = s4[0] # 268개 각 저널별 유사도 점수; test document 1개씩 처리 중이기 때문에 [i]가 아닌 [0] indexing
        
        # 높은 유사도 순으로 후보 저널 정렬0
        best_jj_sim = sorted(dict(zip(journ_index.index, s4)).items(), key=itemgetter(1), reverse=True)

        # 그 중 기존 list of journals에 없는 저널 탐색 (유사도 높은 후보부터)
        new_journ_found = next(filter(if_not_in_r, best_jj_sim), None)
        if new_journ_found != None: # list of journals에 없는 후보 저널 1개를 찾았고,
            if new_journ_found[1] > 0: # 해당 저널의 유사도 점수가 0보다 크다면
                # 후보 저널 추가
                top_journs_added = cur_top_journs + [new_journ_found[0]] # list of journals에 추가
                add_journ_found = True
                return top_journs_added
            else: # 유사도 점수가 0이면, 다음 후보 저널로 탐색 (R2,R3,...)
                continue
        else: # list of journals에 없는 후보 저널이 없다면(모든 후보 저널이 이미 다 list of journals에 있다면)
            continue # 다음 후보 저널로 탐색 (R2,R3,...)
    
    # 모든 후보 저널을(R1,R2,R3,...) 다 탐색했지만, 적합한 후보 저널을 못 찾았다면
    if not add_journ_found:
        top_journs_added = cur_top_journs # 기존 list of journals 그대로 사용
        return top_journs_added

# # 모델링 및 평가
- Best model 사용
- Top-K 저널 추천 및 정확도 평가

In [ ]:
# 추천 모델 평가 지표 함수

# Micro Accuracy
def get_acc_micro(y_pred, y_true):
    return np.mean(np.array([1 if y_true[i] in y_pred[i] else 0 for i in range(len(y_true))])) 

# Macro Accuracy
def get_acc_macro(y_pred, y_true):
    global each_journ_ox, each_journ_mean_acc
    # 저널별로 맞은 것은 1, 틀린 것은 0으로 넣기
    each_journ_ox = collections.defaultdict(list)
    for journ_actual, journs_rec in list(zip(y_true, y_pred)):
        if journ_actual in journs_rec:
            each_journ_ox[journ_actual].append(1)
        else:
            each_journ_ox[journ_actual].append(0)
    # 저널별 평균 accuracy 계산
    each_journ_mean_acc = {key: np.mean(values) for key, values in each_journ_ox.items()} # dictionary comprehension
    # 저널별 평균 accuracy의 전체 평균 계산
    mean_of_mean_acc = np.mean(list(each_journ_mean_acc.values()))
    return mean_of_mean_acc

# MRR
def get_mrr(y_pred, y_true):
    # 각 예측별 reciprocal rank 구하기 (실제 저널이 추천 리스트에서 몇 번째에 위치하는지)
    recip_rank_computed = []
    for journ_actual, journs_rec in list(zip(y_true, y_pred)):
        try:
            # 추천 리스트에 있다면, 위치 번호에 역수 취하기 (앞에 나올수록 좋은 모델이니까)
            recip_rank = 1 / (journs_rec.index(journ_actual) + 1) # 1/(index + 1)
        except:
            # 추천 리스트에 없다면 0
            recip_rank = 0
        recip_rank_computed.append(recip_rank)
    # Mean Reciprocal Rank 계산
    mmr = np.mean(recip_rank_computed)
    return mmr

In [334]:
# 추천 저널 수(K) 설정
# Top-3, Top-5, Top-10, Top-15, Top-20

K_list = [3, 5, 10, 15, 20]

## Phase 1+2+3: Abs + Tt + Kw
- Top-(K-1) 저널 추출 후, 유사 저널 1개 추가

In [21]:
# 기존 모델에서 sum이 아닌 mean 사용한 모델 평가
# 1) 모든 유사도에 대한 mean (O)

tt_kw_exist = True
tt_kw_s = s23

accuracy_results = pd.DataFrame() # 정확도 결과 df

for K in K_list:
    print('Current K:', K)
    
    ## (1차) Test document별 Top-(K-1) 저널
    # Abstract에 대한 doc-to-doc 유사도 합계가 높은 순서로 저널 정렬됨
    # 1차에서 찾을 추천 후보 저널 수: K-1
    first_n = K-1
    top_k_journs = np.array(r)[:, :first_n]

    ## (2차) R-resort : List of Journals 재배치
    # Title and/or keyword가 있다면, test doc vs. train journ의 Title 및 Keyword 유사도 높은 순서로 재정렬
    if tt_kw_exist:
        r_resort = list()
        for i in tqdm(range(len(tt_kw_s)), position=0, leave=True): # 10307
            r_resort.append(resort_r(tt_kw_s, top_k_journs, i))
    # 없다면, Abstract 유사도 기준으로 정렬된 1차 추천 후보 리스트 그대로 사용
    else:
        r_resort = top_k_journs

    ## (3차) 상위 1개 journal-journal 유사도 분석 및 추가 할 수 있는 후보 저널 탐색
    #workers = os.cpu_count() * 2
    workers = -1
    with parallel_backend(backend='threading', n_jobs=workers):
        top_k_r = list(Parallel()(delayed(find_other_sim_journ)(top_journs_list) for top_journs_list in tqdm(r_resort, position=0, leave=True)))    
    
    ## 정확도 평가
    micro_acc = get_acc_micro(top_k_r, test_y) # Top-K Micro Accuracy (전체 Accuracy 합계 / 전체 N)
    macro_acc = get_acc_macro(top_k_r, test_y) # Top-K Macro Accuracy (저널별 평균 accuracy의 평균)
    mrr = get_mrr(top_k_r, test_y) # Top-K MRR (실제 저널이 추천 저널 리스트에 있다면 해당 순위를 반영해 점수 계산)
    top_k_eval = pd.DataFrame([micro_acc, macro_acc, mrr], index=['micro_acc', 'macro_acc', 'mrr'], columns=[f'Top-{K}'])
    
    accuracy_results = pd.concat([accuracy_results,top_k_eval], axis=1)
    
acc_s_curve = accuracy_results.copy()
print(acc_s_curve)
acc_s_curve

Current K: 3


100%|████████████████████████████████████| 10307/10307 [00:42<00:00, 240.35it/s]


Current K: 5


100%|████████████████████████████████████| 10307/10307 [00:38<00:00, 269.77it/s]


Current K: 10


...

Current K: 15


100%|████████████████████████████████████| 10307/10307 [00:39<00:00, 260.45it/s]


Current K: 20


100%|████████████████████████████████████| 10307/10307 [00:39<00:00, 260.13it/s]


,Top-3,Top-5,Top-10,Top-15,Top-20
micro_acc,0.3297,0.4265,0.5540,0.6189,0.6724
macro_acc,0.4199,0.5353,0.6691,0.7396,0.7806
mrr,0.2486,0.2973,0.3450,0.3644,0.3767


In [23]:
print(acc_s_curve)

           Top-3  Top-5  Top-10  Top-15  Top-20
micro_acc 0.3297 0.4265  0.5540  0.6189  0.6724
macro_acc 0.4199 0.5353  0.6691  0.7396  0.7806
mrr       0.2486 0.2973  0.3450  0.3644  0.3767


In [32]:
import pandas as pd

# 결과 csv 저장
acc_s_curve.to_csv('./acc_scurve_model/acc_s_curve.csv', index=True)

# 결과 csv 다시 읽기
# acc_s_curve = pd.read_csv('./acc_scurve_model/acc_s_curve.csv', index_col=0)
# acc_s_curve

# 끝